<a href="https://colab.research.google.com/github/ben14132/ITN-finetuning/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb
!pip install  -q ipywidgets
!pip install -q scipy
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-b9vmiej8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-b9vmiej8
  Resolved https://github.com/huggingface/peft.git to commit ffb8512396e24e9e60e9f1fec02052ac6b407a23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.10.1.dev0-py3-none-any.whl size=199142 sha256=168f6ab6385e059474383f05287b75824796fbb02e5e2114e6746eb3e4a39691
  Stored in directory: /tmp/pip-ephem-wheel-cache-ybpypatp/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.10.0
    Uninstalling peft-0.10.0:
      Successfully uninstalled peft-0.10.0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-584zw4_q
  Running command gi

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import sentencepiece
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig, pipeline
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,use_fast=False,add_eos_token=True) # Set trust_remote_code=True
tokenizer.pad_token_id = 18610

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
PEFT_MODEL = "Ben141/LLM24"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

In [ ]:
def generate_answer(query):
  user_prompt = f"""{query}"""

  final_prompt = f"""
  Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
  ### Instruction:
  The following sentence is in its spoken form, convert it to its written form.

  ### Input:
  {user_prompt}

  ### Response:"""


  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  # encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  # outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
  #                                                                                                                    eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
  #                                                                                                                    temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  # text_output = tokenizer.decode(outputs[0])

  # print(dashline)
  # print(f'ORIGINAL MODEL RESPONSE:\n{text_output}')
  # print(dashline)

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, add_special_tokens = False, \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens = True)
  output_list = peft_text_output.split(":")

  hypothesis = output_list[-1].strip()

  return hypothesis

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("symbols test.xlsx")

In [ ]:
print(df)

                                                input  hypothesis  \
0   my email is first name dot last name at compan...         NaN   
1   the cost of the item is twenty dollars and nin...         NaN   
2   you can find the documentation at h t t p s co...         NaN   
3       his twitter handle is at symbol john q public         NaN   
4                    the formula for water is h two o         NaN   
..                                                ...         ...   
95  our official hashtag for the event is hash tag...         NaN   
96  look for the item that has the tag new arrival...         NaN   
97  you can direct message me on twitter at symbol...         NaN   
98  our website has moved to a new address please ...         NaN   
99  sign up for our webinar by clicking the sign u...         NaN   

                                               output  WER  
0          My email is firstname.lastname@company.com  NaN  
1                      The cost of the item is $2

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.4 MB/s eta 0:00:00


In [ ]:
import jiwer


transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

In [ ]:
for a in range(100):
  input = df.input[a]
  hypothesis = generate_answer(input)
  df.hypothesis[a] = hypothesis
  reference = df.output[a]
  wer = jiwer.wer(
                  reference,
                  hypothesis,
                  truth_transform=transforms,
                  hypothesis_transform=transforms,
              )
  df.WER[a] = wer



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [ ]:
with pd.ExcelWriter("symbols results.xlsx") as writer:
   df.to_excel(writer)